In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import symlib 
import matplotlib.colors as colors
from glob import glob

In [17]:
def save_concentration(base_dir, suite_name, output):
    output_dir = os.path.join(output, "output", suite_name)
    os.makedirs(output_dir, exist_ok=True)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, cvir_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(base_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            cvir = r[0, -1]["cvir"]
            halo_ids.append(halo_idx)
            cvir_list.append(cvir)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "cvir": cvir_list})
    df.to_csv(os.path.join(output_dir, "halo_concentrations.csv"), index=False)
    print(f"[Saved] Concentration CSV to {output_dir}/halo_concentrations.csv")

save_concentration('/Volumes/Atlas/Symphony', 'SymphonyLMC', "/Users/fengbocheng/Projects/Symphony-PPSD")
save_concentration('/Volumes/Atlas/Symphony', 'SymphonyMilkyWay', "/Users/fengbocheng/Projects/Symphony-PPSD")
save_concentration('/Volumes/Expansion/Symphony', 'SymphonyGroup', "/Users/fengbocheng/Projects/Symphony-PPSD")
save_concentration('/Volumes/Atlas/Symphony', 'SymphonyLCluster', "/Users/fengbocheng/Projects/Symphony-PPSD")
# save_concentration('/Volumes/Expansion/Symphony', 'SymphonyCluster', "/Users/fengbocheng/Projects/Symphony-PPSD")

[Saved] Concentration CSV to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLMC/halo_concentrations.csv
[Saved] Concentration CSV to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyMilkyWay/halo_concentrations.csv
[Saved] Concentration CSV to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyGroup/halo_concentrations.csv
[Saved] Concentration CSV to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLCluster/halo_concentrations.csv


In [2]:
def ppsd_profiles(base_dir, suite_name):
    # Define input/output directories
    density_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    velocity_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    output_dir = os.path.join(base_dir, "output", suite_name, "ppsd_profiles")
    os.makedirs(output_dir, exist_ok=True)

    # Collect file names
    density_files = sorted([f for f in os.listdir(density_dir) if f.endswith(".csv")])
    mass_files = sorted([f for f in os.listdir(mass_dir) if f.endswith(".csv")])
    velocity_files = sorted([f for f in os.listdir(velocity_dir) if f.endswith(".csv")])

    # Loop through halos and compute Q_r and Q_tot
    for i, (f_rho, f_mass, f_vel) in enumerate(zip(density_files, mass_files, velocity_files)):
        df_rho = pd.read_csv(os.path.join(density_dir, f_rho))
        df_mass = pd.read_csv(os.path.join(mass_dir, f_mass))
        df_vel = pd.read_csv(os.path.join(velocity_dir, f_vel))

        # Load profile data
        r = df_rho["r_scaled"].values
        rho = df_rho["rho_scaled"].values
        m = df_mass["m_scaled"].values
        sigma_rad = df_vel["sigma_rad_scaled"].values
        sigma_tot = df_vel["sigma_total_scaled"].values

        # Compute pseudo phase-space densities
        with np.errstate(divide="ignore", invalid="ignore"):
            Q_r = np.where(sigma_rad > 0, rho / sigma_rad**3, np.nan)
            Q_tot = np.where(sigma_tot > 0, rho / sigma_tot**3, np.nan)

        # Save to CSV
        df_out = pd.DataFrame({
            "r_scaled": r,
            "m_scaled": m,
            "Q_r": Q_r,
            "Q_tot": Q_tot
        })
        df_out.to_csv(os.path.join(output_dir, f"ppsd_profile_{i}.csv"), index=False)

    print(f"[Saved] PPSD profiles for {suite_name} saved to {output_dir}")


In [18]:
def plot_ppsd_profiles_colored_by_c(base_dir, suite_name):
    
    profile_dir = os.path.join(base_dir, "output", suite_name, "ppsd_profiles")
    output_dir = os.path.join(base_dir, "output", suite_name, "figures")
    os.makedirs(output_dir, exist_ok=True)

    # --- Step 1: Load cvir CSV ---
    cvir_path = os.path.join(base_dir, "output", suite_name, "halo_concentrations.csv")
    cvir_df = pd.read_csv(cvir_path)
    cvir_dict = dict(zip(cvir_df["halo_id"], cvir_df["cvir"]))

    # --- Step 2: Load profiles ---
    files = sorted([f for f in os.listdir(profile_dir) if f.endswith(".csv")])
    ppsd_r, ppsd_tot, mass_profiles, concentrations = [], [], [], []

    for f in files:
        df = pd.read_csv(os.path.join(profile_dir, f))
        r = df["r_scaled"].values
        Q_r = df["Q_r"].values
        Q_tot = df["Q_tot"].values
        m = df["m_scaled"].values
        ppsd_r.append(Q_r)
        ppsd_tot.append(Q_tot)
        mass_profiles.append(m)

        # Use halo ID from filename to look up concentration
        halo_id = int(f.split("_")[-1].split(".")[0])
        cvir = cvir_dict.get(halo_id, np.nan)
        concentrations.append(cvir)

    # Convert to arrays
    ppsd_r = np.array(ppsd_r)
    ppsd_tot = np.array(ppsd_tot)
    mass_profiles = np.array(mass_profiles)
    concentrations = np.array(concentrations)
    n_halos = len(ppsd_r)

    # Compute mean reference curves (Qr_ref, Qtot_ref ∝ r^-1.875)
    mean_Qr = np.nanmean(ppsd_r, axis=0)
    mean_Qtot = np.nanmean(ppsd_tot, axis=0)
    valid_r = ~np.isnan(mean_Qr)
    valid_tot = ~np.isnan(mean_Qtot)

    log_r = np.log(r[valid_r])
    log_Qr = np.log(mean_Qr[valid_r])
    A_r = np.exp(np.mean(log_Qr + 1.875 * log_r))
    log_Qtot = np.log(mean_Qtot[valid_tot])
    A_tot = np.exp(np.mean(log_Qtot + 1.875 * np.log(r[valid_tot])))

    ref_curve_r = A_r * r**(-1.875)
    ref_curve_tot = A_tot * r**(-1.875)

    # Compute residuals for each halo
    residuals_r, residuals_tot = [], []
    for i in range(n_halos):
        Qr = ppsd_r[i]
        Qt = ppsd_tot[i]
        res_r = np.full_like(Qr, np.nan)
        res_t = np.full_like(Qt, np.nan)
        idx_r = ~np.isnan(Qr)
        idx_t = ~np.isnan(Qt)
        if np.any(idx_r):
            res_r[idx_r] = np.log10(Qr[idx_r]) - np.log10(ref_curve_r[idx_r])
        if np.any(idx_t):
            res_t[idx_t] = np.log10(Qt[idx_t]) - np.log10(ref_curve_tot[idx_t])
        residuals_r.append(res_r)
        residuals_tot.append(res_t)

    # Convert residuals to arrays and compute stats
    residuals_r = np.array(residuals_r)
    residuals_tot = np.array(residuals_tot)
    mean_res_r = np.nanmean(residuals_r, axis=0)
    std_res_r = np.nanstd(residuals_r, axis=0)
    mean_res_tot = np.nanmean(residuals_tot, axis=0)
    std_res_tot = np.nanstd(residuals_tot, axis=0)

    # Set up colormap for cvir
    cmap = cm.viridis
    norm = plt.Normalize(vmin=np.nanmin(concentrations), vmax=np.nanmax(concentrations))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # --- Figure 1: Q vs r ---
    fig1, ax1 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        ax1[0].plot(r, ppsd_r[i], color=cmap(norm(concentrations[i])), lw=0.7)
        ax1[1].plot(r, ppsd_tot[i], color=cmap(norm(concentrations[i])), lw=0.7)
    ax1[0].set(title=r"$Q_r$ vs $r$", xscale="log", yscale="log",
               xlabel=r"$r / r_{\rm vir}$", ylabel=r"$Q_r$")
    ax1[1].set(title=r"$Q_{\rm tot}$ vs $r$", xscale="log", yscale="log",
               xlabel=r"$r / r_{\rm vir}$", ylabel=r"$Q_{\rm tot}$")
    for ax in ax1: ax.grid(True, which="both", linestyle=":")
    fig1.colorbar(sm, ax=ax1.ravel().tolist(), shrink=0.9).set_label(r"$c_{\rm vir}$")
    fig1.suptitle(f"PPSD vs r (Colored by c): {suite_name}")
    fig1.savefig(os.path.join(output_dir, "PPSD_vs_r (colored by c).png"))
    plt.show()
    plt.close(fig1)

    # --- Figure 2: Q vs M(<r) ---
    fig2, ax2 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        ax2[0].plot(mass_profiles[i], ppsd_r[i], color=cmap(norm(concentrations[i])), lw=0.7)
        ax2[1].plot(mass_profiles[i], ppsd_tot[i], color=cmap(norm(concentrations[i])), lw=0.7)
    ax2[0].set(title=r"$Q_r$ vs $M(<r)$", xscale="log", yscale="log",
               xlabel=r"$M(<r)/M_{\rm vir}$", ylabel=r"$Q_r$")
    ax2[1].set(title=r"$Q_{\rm tot}$ vs $M(<r)$", xscale="log", yscale="log",
               xlabel=r"$M(<r)/M_{\rm vir}$", ylabel=r"$Q_{\rm tot}$")
    for ax in ax2: ax.grid(True, which="both", linestyle=":")
    fig2.colorbar(sm, ax=ax2.ravel().tolist(), shrink=0.9).set_label(r"$c_{\rm vir}$")
    fig2.suptitle(f"PPSD vs Mass (Colored by c): {suite_name}")
    fig2.savefig(os.path.join(output_dir, "PPSD_vs_mass (colored by c).png"))
    plt.show()
    plt.close(fig2)

    # --- Figure 3: Residuals ---
    fig3, ax3 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        ax3[0].plot(r, residuals_r[i], color=cmap(norm(concentrations[i])), lw=0.7)
        ax3[1].plot(r, residuals_tot[i], color=cmap(norm(concentrations[i])), lw=0.7)
    # Add mean ± 1σ lines
    ax3[0].plot(r, mean_res_r, 'k-', lw=1)
    ax3[0].fill_between(r, mean_res_r - std_res_r, mean_res_r + std_res_r, color='gray', alpha=0.3)
    ax3[0].axhline(0, color='r', linestyle='--', lw=1)
    ax3[0].set(title=r"$\log_{10}(Q_r/Q_{\rm ref})$ vs $r / r_{\rm vir}$",
               xlabel=r"$r / r_{\rm vir}$", ylabel=r"$\log_{10}(Q_r/Q_{\rm ref})$",
               xscale="log", ylim=(-1.5, 1.5))
    ax3[1].plot(r, mean_res_tot, 'k-', lw=1)
    ax3[1].fill_between(r, mean_res_tot - std_res_tot, mean_res_tot + std_res_tot, color='gray', alpha=0.3)
    ax3[1].axhline(0, color='r', linestyle='--', lw=1)
    ax3[1].set(title=r"$\log_{10}(Q_{\rm tot}/Q_{\rm ref})$ vs $r / r_{\rm vir}$",
               xlabel=r"$r / r_{\rm vir}$", ylabel=r"$\log_{10}(Q_{\rm tot}/Q_{\rm ref})$",
               xscale="log", ylim=(-1.5, 1.5))
    for ax in ax3: ax.grid(True, which="both", linestyle=":")
    fig3.colorbar(sm, ax=ax3.ravel().tolist(), shrink=0.9).set_label(r"$c_{\rm vir}$")
    fig3.suptitle(f"PPSD Residuals (Colored by c): {suite_name}")
    fig3.savefig(os.path.join(output_dir, "PPSD_residuals (colored by c).png"))
    plt.show()
    plt.close(fig3)

    print("[Saved] Colored PPSD Figures")

In [21]:
def plot_density_colored_by_c(base_dir, suite_name): 
   # Load concentration CSV
    cvir_path = os.path.join(base_dir, "output", suite_name, "halo_concentrations.csv")
    if not os.path.exists(cvir_path):
        print(f"[Error] Concentration file not found: {cvir_path}")
        return
    cvir_df = pd.read_csv(cvir_path)
    cvir_dict = dict(zip(cvir_df["halo_id"], cvir_df["cvir"]))

    # Load density profile CSVs
    profile_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    file_list = sorted(glob(os.path.join(profile_dir, "halo_*_profile.csv")))
    n_halos = len(file_list)
    if n_halos == 0:
        print("[Error] No profile files found.")
        return

    # Prepare arrays
    all_profiles = []
    r_scaled_list = []
    c_vir_list = []

    for file in file_list:
        df = pd.read_csv(file)
        df["rho_r2"] = df["rho_scaled"] * df["r_scaled"]**2
        all_profiles.append(df["rho_r2"].values)
        r_scaled_list.append(df["r_scaled"].values)

        halo_id = int(file.split("_")[-2])  # Assumes name like halo_005_profile.csv
        c_vir = cvir_dict.get(halo_id, np.nan)
        c_vir_list.append(c_vir)

    c_vir_arr = np.array(c_vir_list)
    cmap = cm.viridis
    norm = plt.Normalize(vmin=np.nanmin(c_vir_arr), vmax=np.nanmax(c_vir_arr))
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    # Create output directory
    fig_dir = os.path.join(base_dir, "output", suite_name, "figures")
    os.makedirs(fig_dir, exist_ok=True)

    # -------- Plot: All profiles colored by c_vir --------
    fig, ax = plt.subplots(figsize=(6.5, 5), dpi=500)
    for rho_r2, r, c in zip(all_profiles, r_scaled_list, c_vir_arr):
        ax.plot(r, rho_r2, color=cmap(norm(c)), lw=0.7, alpha=0.8)

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel(r"$r / r_{\mathrm{vir}}$")
    ax.set_ylabel(r"$(\rho/\bar{\rho}_m) \cdot (r/r_{\mathrm{vir}})^2$")
    ax.grid(True, which="both", linestyle=":")
    ax.set_title(f"Density Profiles Colored by $c$ ({suite_name})")

    cbar = fig.colorbar(sm, ax=ax, pad=0.01)
    cbar.set_label(r"$c_{\rm vir}$")
    fig.tight_layout()
    fig.savefig(os.path.join(fig_dir, "density_profiles (colored by c).png"))
    plt.show()
    plt.close(fig)

    print(f"[Done] Colored density profiles saved to {fig_dir}")


In [39]:
def plot_velocity_colored_by_c(base_dir, suite_name):
    input_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    output_dir = os.path.join(base_dir, "output", suite_name, "figures")
    os.makedirs(output_dir, exist_ok=True)

    # Load cvir CSV
    cvir_path = os.path.join(base_dir, "output", suite_name, "halo_concentrations.csv")
    if not os.path.exists(cvir_path):
        print(f"[Error] Concentration file not found: {cvir_path}")
        return
    cvir_df = pd.read_csv(cvir_path)
    cvir_dict = dict(zip(cvir_df["halo_id"], cvir_df["cvir"]))

    # Load velocity profiles
    files = sorted([f for f in os.listdir(input_dir) if f.endswith(".csv")])
    if not files:
        print("[Error] No velocity profile files found.")
        return

    r = pd.read_csv(os.path.join(input_dir, files[0]))["r_scaled"].values
    sigma_rad_all, sigma_tan_all, sigma_total_all, beta_all, cvir_all = [], [], [], [], []

    for f in files:
        df = pd.read_csv(os.path.join(input_dir, f))
        sigma_rad_all.append(df["sigma_rad_scaled"].values)
        sigma_tan_all.append(df["sigma_tan_scaled"].values)
        sigma_total_all.append(df["sigma_total_scaled"].values)
        beta_all.append(df["beta"].values)

        # Extract halo ID from filename
        import re
        match = re.search(r"halo_(\d+)", f)
        halo_id = int(match.group(1))
        cvir = cvir_dict.get(halo_id, np.nan)
        cvir_all.append(cvir)

    # Convert and normalize colors
    cvir_arr = np.array(cvir_all)
    cmap = cm.viridis
    norm = plt.Normalize(vmin=np.nanmin(cvir_arr), vmax=np.nanmax(cvir_arr))
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    # Prepare plotting arrays
    data_arrs = [sigma_rad_all, sigma_tan_all, sigma_total_all, beta_all]
    titles = [r"$\sigma_{\mathrm{rad}} / V_{\mathrm{vir}}$", 
              r"$\sigma_{\mathrm{tan}} / V_{\mathrm{vir}}$",
              r"$\sigma_{\mathrm{total}} / V_{\mathrm{vir}}$",
              r"$\beta$"]

    # Plot 4 stacked velocity-related profiles
    fig, axes = plt.subplots(4, 1, figsize=(7, 10), sharex=True, dpi=500, constrained_layout=True)
    for i in range(4):
        for y, c in zip(data_arrs[i], cvir_arr):
            axes[i].plot(r, y, color=cmap(norm(c)), lw=0.7, alpha=0.8)
        axes[i].set_ylabel(titles[i])
        axes[i].set_xscale("log")
        axes[i].grid(True, which="both", linestyle=":")
        
    axes[-1].set_ylim(-1, 1)
    axes[-1].set_xlabel(r"$r / r_{\mathrm{vir}}$")
    fig.suptitle(f"Velocity Profiles Colored by $c$ ({suite_name})", fontsize=14)
    cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), pad=0.01)
    cbar.set_label(r"$c_{\rm vir}$")

    fig.savefig(os.path.join(output_dir, "velocity_profiles (colored_by_c).png"))
    plt.show()
    plt.close(fig)

    print(f"[Saved] Velocity profiles with concentration coloring saved for {suite_name}.")

In [6]:
import pynumdiff
import pynumdiff.optimize

def get_diff_and_optimize_funcs(method):
    submodules = [
        'kalman_smooth',
        'smooth_finite_difference',
        'finite_difference',
        'total_variation_regularization',
        'linear_model'
    ]
    for submod in submodules:
        try:
            mod_optimize = getattr(pynumdiff.optimize, submod)
            mod_diff = getattr(pynumdiff, submod)
            if hasattr(mod_optimize, method) and hasattr(mod_diff, method):
                return getattr(mod_diff, method), getattr(mod_optimize, method)
        except AttributeError:
            continue
    raise ValueError(f"Method '{method}' not found in any submodule.")

def fit_and_save_ppsd_slopes(base_dir, suite_name, method='constant_jerk', tvgamma=None):
    # Paths
    density_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    velocity_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    slope_r_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_r")
    slope_m_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_m")
    os.makedirs(slope_r_dir, exist_ok=True)
    os.makedirs(slope_m_dir, exist_ok=True)

    density_files = sorted([f for f in os.listdir(density_dir) if f.endswith(".csv")])
    velocity_files = sorted([f for f in os.listdir(velocity_dir) if f.endswith(".csv")])
    mass_files = sorted([f for f in os.listdir(mass_dir) if f.endswith(".csv")])
    n_halos = len(density_files)

    def fit_derivative(y, dt):
        try:
            diff_func, optimize_func = get_diff_and_optimize_funcs(method)
            kwargs = {'tvgamma': tvgamma} if 'tvgamma' in optimize_func.__code__.co_varnames else {}
            params, _ = optimize_func(y, dt, **kwargs)
            _, dydx = diff_func(y, dt, params)
            return dydx
        except Exception as e:
            print(f"{method} derivative fit failed: {e}")
            return None

    for i in range(n_halos):
        try:
            df_rho = pd.read_csv(os.path.join(density_dir, density_files[i]))
            df_vel = pd.read_csv(os.path.join(velocity_dir, velocity_files[i]))
            df_mass = pd.read_csv(os.path.join(mass_dir, mass_files[i]))
        except Exception as e:
            print(f"[Halo {i}] loading profiles failed: {e}")
            continue

        r = df_rho["r_scaled"].values
        m = df_mass["m_scaled"].values
        rho = df_rho["rho_scaled"].values
        sigma_tot = df_vel["sigma_total_scaled"].values
        sigma_rad = df_vel["sigma_rad_scaled"].values

        dt_r = np.diff(np.log10(r)).mean()
        dt_m = np.diff(np.log10(m)).mean()

        log_rho = np.log10(rho)
        log_sigma_tot = np.log10(sigma_tot)
        log_sigma_rad = np.log10(sigma_rad)

        drho_dlogr = fit_derivative(log_rho, dt_r)
        dsigma_tot_dlogr = fit_derivative(log_sigma_tot, dt_r)
        dsigma_rad_dlogr = fit_derivative(log_sigma_rad, dt_r)

        drho_dlogm = fit_derivative(log_rho, dt_m)
        dsigma_tot_dlogm = fit_derivative(log_sigma_tot, dt_m)
        dsigma_rad_dlogm = fit_derivative(log_sigma_rad, dt_m)

        if None in [drho_dlogr, dsigma_tot_dlogr, dsigma_rad_dlogr, drho_dlogm, dsigma_tot_dlogm, dsigma_rad_dlogm]:
            print(f"[Halo {i}] derivative fitting failed, skipping")
            continue

        slope_Q_tot_r = drho_dlogr - 3 * dsigma_tot_dlogr
        slope_Q_rad_r = drho_dlogr - 3 * dsigma_rad_dlogr
        slope_Q_tot_m = drho_dlogm - 3 * dsigma_tot_dlogm
        slope_Q_rad_m = drho_dlogm - 3 * dsigma_rad_dlogm

        df_r = pd.DataFrame({"r_scaled": r, "slope_Q_r": slope_Q_rad_r, "slope_Q_tot": slope_Q_tot_r})
        df_m = pd.DataFrame({"m_scaled": m, "slope_Q_r": slope_Q_rad_m, "slope_Q_tot": slope_Q_tot_m})

        df_r.to_csv(os.path.join(slope_r_dir, f"slope_profile_r_{i}.csv"), index=False)
        df_m.to_csv(os.path.join(slope_m_dir, f"slope_profile_m_{i}.csv"), index=False)


In [7]:
def plot_ppsd_slope_colored_by_c(base_dir, suite_name):
    slope_r_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_r")
    slope_m_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_m")
    output_dir = os.path.join(base_dir, "output", suite_name, "figures")
    os.makedirs(output_dir, exist_ok=True)

    slope_r_files = sorted([f for f in os.listdir(slope_r_dir) if f.endswith(".csv")])
    slope_m_files = sorted([f for f in os.listdir(slope_m_dir) if f.endswith(".csv")])
    n_halos = len(slope_r_files)

    concentrations = []
    for i in range(n_halos):
        sim_dir = symlib.get_host_directory(base_dir, suite_name, i)
        try:
            r_data, _ = symlib.read_rockstar(sim_dir)
            cvir_val = r_data[0, -1]["cvir"]
        except Exception as e:
            print(f"[Halo {i}] concentration failed: {e}")
            cvir_val = np.nan
        concentrations.append(cvir_val)
    concentrations = np.array(concentrations)

    cmap = cm.viridis
    norm = colors.Normalize(vmin=np.nanmin(concentrations), vmax=np.nanmax(concentrations))
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    fig1, axes1 = plt.subplots(1, 2, figsize=(13, 5), dpi=500, constrained_layout=True)
    fig2, axes2 = plt.subplots(1, 2, figsize=(13, 5), dpi=500, constrained_layout=True)
    ax_slope_tot, ax_slope_rad = axes1
    ax_slope_tot_m, ax_slope_rad_m = axes2

    for i in range(n_halos):
        try:
            df_r = pd.read_csv(os.path.join(slope_r_dir, slope_r_files[i]))
            df_m = pd.read_csv(os.path.join(slope_m_dir, slope_m_files[i]))
        except:
            continue

        r = df_r["r_scaled"].values
        m = df_m["m_scaled"].values
        slope_Q_tot_r = df_r["slope_Q_tot"].values
        slope_Q_rad_r = df_r["slope_Q_r"].values
        slope_Q_tot_m = df_m["slope_Q_tot"].values
        slope_Q_rad_m = df_m["slope_Q_r"].values

        color = cmap(norm(concentrations[i])) if np.isfinite(concentrations[i]) else "gray"
        ax_slope_tot.plot(r, slope_Q_tot_r, color=color, alpha=0.5, lw=0.5)
        ax_slope_rad.plot(r, slope_Q_rad_r, color=color, alpha=0.5, lw=0.5)
        ax_slope_tot_m.plot(m, slope_Q_tot_m, color=color, alpha=0.5, lw=0.5)
        ax_slope_rad_m.plot(m, slope_Q_rad_m, color=color, alpha=0.5, lw=0.5)

    for ax, label, xscale in zip(
        [ax_slope_tot, ax_slope_rad],
        [r"$\mathrm{d}\log Q / \mathrm{d}\log r$", r"$\mathrm{d}\log Q_r / \mathrm{d}\log r$"],
        ["log", "log"]):
        ax.set_xlabel(r"$r / R_\mathrm{vir}$")
        ax.set_ylabel(label)
        ax.set_xscale(xscale)
        ax.set_ylim(-4, 1)
        ax.axhline(-1.875, color='k', ls='--', lw=0.7)
        ax.grid(True, which="both", linestyle=":")

    for ax, label in zip([ax_slope_tot_m, ax_slope_rad_m],
                         [r"$\mathrm{d}\log Q / \mathrm{d}\log M$", r"$\mathrm{d}\log Q_r / \mathrm{d}\log M$"]):
        ax.set_xlabel(r"$M(<r) / M_\mathrm{vir}$")
        ax.set_ylabel(label)
        ax.set_xscale("log")
        ax.set_ylim(-4, 1)
        ax.grid(True, which="both", linestyle=":")

    fig1.colorbar(sm, ax=axes1.ravel().tolist(), shrink=0.9).set_label(r"$c_\mathrm{vir}$")
    fig2.colorbar(sm, ax=axes2.ravel().tolist(), shrink=0.9).set_label(r"$c_\mathrm{vir}$")

    fig1.savefig(os.path.join(output_dir, f"PPSD_slopes_r_colored_by_c.png"))
    fig2.savefig(os.path.join(output_dir, f"PPSD_slopes_m_colored_by_c.png"))
    plt.show()
    plt.close(fig1)
    plt.close(fig2)

    print("[Saved] slope plots colored by c")

In [ ]:
base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = [
    "SymphonyLMC",
    "SymphonyMilkyWay",
    "SymphonyGroup",
    "SymphonyLCluster",
]

for suite in suite_names:
    ppsd_profiles(base_dir, suite)


[Saved] PPSD profiles for SymphonyLMC saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLMC/ppsd_profiles
[Saved] PPSD profiles for SymphonyMilkyWay saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyMilkyWay/ppsd_profiles
[Saved] PPSD profiles for SymphonyGroup saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyGroup/ppsd_profiles
[Saved] PPSD profiles for SymphonyLCluster saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLCluster/ppsd_profiles


In [ ]:
for suite in suite_names:
    plot_ppsd_profiles_colored_by_c(base_dir, suite)

In [ ]:
for suite in suite_names:
    plot_density_colored_by_c(base_dir, suite)

In [ ]:
for suite in suite_names:
    plot_velocity_colored_by_c(base_dir, suite)

In [ ]:
for suite in suite_names:
    fit_and_save_ppsd_slopes(base_dir, suite)

In [ ]:
for suite in suite_names:
    plot_ppsd_slope_colored_by_c(base_dir, suite)